# ex_lab-09-7-sigmoid_back_prop

LaTeX:
  \begin{align}
    E_0 &= mc^2                              \\
    E &= \frac{mc^2}{\sqrt{1-\frac{v^2}{c^2}}} \\
    A_0 \\
    \TeX\
  \end{align}
  

 
In this file, we will implement back propagations by hands
We will use the Sigmoid Cross Entropy loss function.
This is equivalent to tf.nn.sigmoid_softmax_with_logits(logits, labels)

# [References]
> 1. [Tensorflow Document (tf.nn.sigmoid_softmax_with_logits)](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)
> 2. [Neural Net Backprop in one slide! by Sung Kim](https://docs.google.com/presentation/d/1_ZmtfEjLmhbuM_PqbDYMXXLAqeWN0HwuhcSKnUQZ6MM/edit#slide=id.g1ec1d04b5a_1_83)
> 3. [Back Propagation with Tensorflow by Dan Aloni](http://blog.aloni.org/posts/backprop-with-tensorflow/)
> 4. [Yes you should understand backprop by Andrej Karpathy](https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b#.cockptkn7)

# [Network Architecture]

\begin{align}
 Input: x \\
 Layer1: {x } W + b \\
 Output\ layer = σ(Layer1) \\
 Loss\_i &= - y * log(σ(Layer1)) - (1 - y) * log(1 - σ(Layer1)) \\
 Loss &= tf.reduce\_sum(Loss\_i)
 \end{align}
 
 We want to compute that  
 \begin{align}
 \frac{dLoss}{dW} &= ??? \\
 \frac{dLoss}{db} &= ??? \\
  \end{align}  
 please read "Neural Net Backprop in one slide!" for deriving formulas

 
 


In [16]:
import os
# silence INFO logs set it to 1
# filter out WARNING set it to 2 
# silence ERROR logs (not recommended) set it to 3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np

print('tensorflow version: {0}'.format(tf.__version__))
print('numpy version: {0}'.format(np.__version__))

tensorflow version: 1.2.0
numpy version: 1.13.0


# for reproducibility

In [17]:
tf.set_random_seed(777)  # for reproducibility


# Predicting animal type based on various features

In [18]:
# Predicting animal type based on various features
xy = np.loadtxt('./data_file/data-04-zoo.csv', delimiter=',', dtype=np.float32)
X_data = xy[:, 0:-1]
N = X_data.shape[0]
y_data = xy[:, [-1]]



# y_data has labels from 0 ~ 6

In [19]:
# y_data has labels from 0 ~ 6
print("y has one of the following values")
print(np.unique(y_data))



y has one of the following values
[ 0.  1.  2.  3.  4.  5.  6.]


#  X_data.shape , y_data.shape
> X_data.shape = (101, 16) => 101 samples, 16 features <br />
> y_data.shape = (101, 1)  => 101 samples, 1 label

In [20]:
# X_data.shape = (101, 16) => 101 samples, 16 features
# y_data.shape = (101, 1)  => 101 samples, 1 label
print("Shape of X data: ", X_data.shape)
print("Shape of y data: ", y_data.shape)



Shape of X data:  (101, 16)
Shape of y data:  (101, 1)


# Init. variables

In [21]:
nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

target = tf.one_hot(y, nb_classes)  # one hot
target = tf.reshape(target, [-1, nb_classes])
target = tf.cast(target, tf.float32)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')




# sigma Function
>  sigmoid function <br />
> σ(x) = 1 / (1 + exp(-x))

In [22]:
def sigma(x):
    # sigmoid function
    # σ(x) = 1 / (1 + exp(-x))
    return 1. / (1. + tf.exp(-x))



# sigma_prime Function:
> derivative of the sigmoid function <br />
> σ'(x) = σ(x) * (1 - σ(x))

In [23]:
def sigma_prime(x):
    # derivative of the sigmoid function
    # σ'(x) = σ(x) * (1 - σ(x))
    return sigma(x) * (1. - sigma(x))



# Forward propagtion

In [24]:
# Forward propagtion
layer_1 = tf.matmul(X, W) + b
y_pred = sigma(layer_1)



# Loss Function (end of forwad propagation)

In [25]:
# Loss Function (end of forwad propagation)
loss_i = - target * tf.log(y_pred) - (1. - target) * tf.log(1. - y_pred)
loss = tf.reduce_sum(loss_i)



# Dimension Check

In [26]:
# Dimension Check
assert y_pred.shape.as_list() == target.shape.as_list()




# Back prop (chain rule)
## How to derive? please read "Neural Net Backprop in one slide!"

In [27]:
# Back prop (chain rule)
# How to derive? please read "Neural Net Backprop in one slide!"
d_loss = (y_pred - target) / (y_pred * (1. - y_pred) + 1e-7)
d_sigma = sigma_prime(layer_1)
d_layer = d_loss * d_sigma
d_b = d_layer
d_W = tf.matmul(tf.transpose(X), d_layer)



# Updating network using gradients

In [28]:
# Updating network using gradients
learning_rate = 0.01
train_step = [
    tf.assign(W, W - learning_rate * d_W),
    tf.assign(b, b - learning_rate * tf.reduce_sum(d_b)),
]



# Prediction and Accuracy

In [ ]:
# Prediction and Accuracy
prediction = tf.argmax(y_pred, 1)
acct_mat = tf.equal(tf.argmax(y_pred, 1), tf.argmax(target, 1))
acct_res = tf.reduce_mean(tf.cast(acct_mat, tf.float32))



# Launch graph

In [ ]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(500):
        sess.run(train_step, feed_dict={X: X_data, y: y_data})

        if step % 10 == 0:
            # Within 300 steps, you should see an accuracy of 100%
            step_loss, acc = sess.run([loss, acct_res], feed_dict={
                                      X: X_data, y: y_data})
            print("Step: {:5}\t Loss: {:10.5f}\t Acc: {:.2%}" .format(
                step, step_loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: X_data})
    for p, y in zip(pred, y_data):
        msg = "[{}]\t Prediction: {:d}\t True y: {:d}"
        print(msg.format(p == int(y[0]), p, int(y[0])))

"""
Output Example

Step:     0      Loss:  453.74799        Acc: 38.61%
Step:    20      Loss:   95.05664        Acc: 88.12%
Step:    40      Loss:   66.43570        Acc: 93.07%
Step:    60      Loss:   53.09288        Acc: 94.06%
...
Step:   290      Loss:   18.72972        Acc: 100.00%
Step:   300      Loss:   18.24953        Acc: 100.00%
Step:   310      Loss:   17.79592        Acc: 100.00%
...
[True]   Prediction: 0   True y: 0
[True]   Prediction: 0   True y: 0
[True]   Prediction: 3   True y: 3
[True]   Prediction: 0   True y: 0
...
"""
